In [ ]:
%pip install usf_fabric_monitoring

In [ ]:
from usf_fabric_monitoring.core.pipeline import MonitorHubPipeline
import os

In [ ]:
# Configuration
DAYS_TO_ANALYZE = 90
OUTPUT_DIR = "/lakehouse/default/Files/monitor_hub_analysis" # Adjust for Fabric Lakehouse path if needed
# Or use local path for testing
# OUTPUT_DIR = "exports/monitor_hub_analysis"

In [ ]:
pipeline = MonitorHubPipeline(OUTPUT_DIR)
results = pipeline.run_complete_analysis(days=DAYS_TO_ANALYZE)
pipeline.print_results_summary(results)